In [ ]:
!pip install sentence_transformers

### 모듈 임포트

In [ ]:
import os
from zipfile import ZipFile
import re
import pickle
import random
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import math
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sklearn.metrics.pairwise import cosine_similarity

### 데이터 압축 해제

In [ ]:
data_path = '/content/drive/MyDrive/data/json_seperatly-saved_in_csv1/json_data.zip'
extract_path = '/content/data'

with ZipFile(data_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

### json_list 변환

In [ ]:
folder_path = '/content/data/json_data'

file_names = os.listdir(folder_path)

data_list = []
for file_name in file_names:
  if file_name.startswith("processed_") and file_name.endswith(".pickle"):
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'rb') as file:
      data = pickle.load(file)
      data_list.append(data)

### text_list 변환

In [ ]:
text_list = []
for i in range(len(data_list)):
  text = ""
  for j in range(len(data_list[i]['form'])):
    if type(data_list[i]['form'][j]['text']) == str:
      text += " " + data_list[i]['form'][j]['text']
  text_list.append(text)

### keyword_list 변환

In [ ]:
def is_valid_keyword(keyword):
  if keyword.isdigit():
    return False
  return True

keyword_list = []
keyword_vocab = {}
for i in range(len(data_list)):
  keyword = data_list[i]['keyword']
  keyword = [word for word in keyword if is_valid_keyword(word)]
  keyword_list.append(" ".join(keyword))

### pair 생성 함수

In [ ]:
def get_positive_pairs(text, keyword):
  return (text, keyword)

def get_negative_pairs(text, keyword):
    keyword_set = set(keyword.split())
    negative_keyword = random.choice(keyword_list)

    while set(negative_keyword.split()).intersection(keyword_set):
        negative_keyword = random.choice(keyword_list)

    return (text, negative_keyword)

### positive pairs, negative pairs 정의

In [ ]:
positive_pairs = []
negative_pairs = []

for i in tqdm(range(len(data_list))):
  text = text_list[i]
  keyword = keyword_list[i]
  positive_pairs.append(get_positive_pairs(text, keyword))
  negative_pairs.append(get_negative_pairs(text, keyword))

100%|██████████| 29590/29590 [00:00<00:00, 86019.68it/s]


### Concatenate하여 pairs, labels 정의

In [ ]:
pairs = positive_pairs + negative_pairs
labels = [1]*len(positive_pairs) + [0]*len(negative_pairs)

In [ ]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

sample_data = list(zip(pairs, labels))

train_data, test_data = train_test_split(sample_data, test_size=0.05, random_state=42)

train_examples = [InputExample(texts=[data[0][0], data[0][1]], label=float(data[1])) for data in train_data]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

train_loss = losses.CosineSimilarityLoss(model)

num_epochs = 5
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=num_epochs, warmup_steps=warmup_steps)
model.save('sbert_keyword_extractor_2023_07_05')

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3514 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3514 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3514 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3514 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3514 [00:00<?, ?it/s]

In [ ]:
keyword_embedding_list = []

for keyword in tqdm(keyword_list):
    keyword_embedding_list.append((keyword, model.encode(keyword)))

100%|██████████| 29590/29590 [03:47<00:00, 129.97it/s]


In [ ]:
print(test_data) ((a, b), 0)

[((' It is an innovative\xa0\nbusiness. 2035 비즈니스 제안서 Business mail miridih@miridih.com Business proposal', '온라인셀러 1인가구 홍보 제품정보 스토어팜 생활용품 패브릭 회색 원목가구 상세페이지 가구 의자 소파 모던'), 0), ((' MIRI-\nFLOWER\nSTUDIO "꽃과 함께 하는 힐링데이!\n꽃을 만지며 지친일상을 위로하는\n시간을 가져보세요." Miri Flower 원데이\n클래스 Vase\nFlower Box\nVase & Oasis 수강안내\n바로가기', '수면습관 심플 목록 ppt 순수한 모던 라인 시민교양 의료 노랑 보건 산뜻한 캐릭터 점선 아기자기한 강의 아이콘 하늘색 프레젠테이션 표 일러스트 미니멀 불면증 컬러풀 복지 그래프 마인드맵'), 0), ((' 사업분야에 대한 설명을 적어주세요.\n사용된\xa0서체는 프리텐다드,\xa0크기는 14,\xa0행간은\n1.6입니다. 사업분야에 대한 설명을 적어주세요. Miri Company leads the business.', '고구마 지자체 소상공인 특산물 기획안 축제 갈색 농촌 지역 ppt'), 0), (('', '인테리어 홈인테리어 매트리스 침구 침구구독 침대 홈케어 이불 침실 가구 베게 구독'), 1), ((' 궁금한 점은 질문해 주세요. Thank you 기획팀 ㅣ 김미리 팀장, 이미리 대리', '살림 지혜 식자재 곰팡이 식품보관 꿀팁 식품섭취'), 0), ((' CHAPTER.05 " 마지막 소개,\n\xa0 \xa0 한마디를 남길게요! " 제 소개를 봐주셔서 감사드려요. 마지막 소개 한마디를 남깁니다! 만나서 반가워요! contact: mirimiri@miricanvas.com 성장에 목마른 분들과 함께,\n좋은 영감 주고받으며 꿈을 키우고 싶어요!', '게임 게임그래픽 강연 비즈니스 스토리텔링'), 0), ((' CLASSIFICATION 꽃과 허브의 화훼분류법 관화식물 관엽식물 열매식물 01 여기에

In [ ]:
positive_pairs_test = []
negative_pairs_test = []

test_pair = list(map(lambda x: x[0], test_data))

In [ ]:
for i in tqdm(range(len(test_pair))):
  text = text_list[i]
  keyword = keyword_list[i]
  positive_pairs_test.append(get_positive_pairs(text, keyword))
  negative_pairs_test.append(get_negative_pairs(text, keyword))

100%|██████████| 2959/2959 [00:00<00:00, 87704.28it/s]


In [ ]:
sum = 0
for a, b in tqdm(negative_pairs_test):
  a = model.encode(a)
  b = model.encode(b)
  score = cosine_similarity([a], [b])[0][0]
  sum += score
print("Negative:", sum / len(negative_pairs_test))
sum = 0
for a, b in tqdm(positive_pairs_test):
  a = model.encode(a)
  b = model.encode(b)
  score = cosine_similarity([a], [b])[0][0]
  sum += score
print("Positive:", sum / len(positive_pairs_test))

100%|██████████| 2959/2959 [00:50<00:00, 58.62it/s]


Negative: 0.07940845455677381


100%|██████████| 2959/2959 [00:50<00:00, 58.14it/s]

Positive: 0.903695783768908


In [ ]:
keyword_embedding_set = []
keyword_set = list(set(keyword_list))

for keyword in tqdm(keyword_set):
    keyword_embedding_set.append((keyword, model.encode(keyword)))

100%|██████████| 7453/7453 [01:00<00:00, 122.35it/s]


In [ ]:
def get_top_keyword(sentence, top_k = 3):
    sentence_embedding = model.encode(sentence)
    keyword_score_list = []
    for keyword, keyword_embedding in keyword_embedding_set:
        score = cosine_similarity([sentence_embedding], [keyword_embedding])[0][0]
        keyword_score_list.append((keyword, score))
    keyword_score_list.sort(key=lambda x: x[1], reverse=True)
    return keyword_score_list[:top_k]

In [ ]:
sentence = '''
중간중간 ###### 오타가 있ㅁ는 레입아웃에 대한 시험입닏.
#@!@ 주제는 수영장 ##파티 이#며 이
'''
get_top_keyword(sentence, 10)

[('도형의 친환경 제로웨이스트 기부 비비드 행사 에코 포스터 주민센터 공지 동네 바다식목일 바자회 이웃 식목일 미역 파스텔 환경 추상적인 동사무소 산호초 일러스트 산호 해초 복지 나눔 지구 말미잘',
  0.87040865),
 ('제로웨이스트캠페인 캠페인기획서 제로웨이스트 기획서 기업기획서', 0.85723436),
 ('연탄나눔 봉사활동 자원봉사 포스터', 0.8533262),
 ('캠페인 포스터 조직문화 직장 계몽 사내캠페인 문화 기업', 0.8466792),
 ('모집 자연 플라스틱 환경보호 지원 사랑 어린이 봉사 환경 보호 봉사단 부모 스티커', 0.8376261),
 ('정부 지자체 도움 행정 소외 이웃 지원 사랑 하트 사업 복지 희망 사각지대 주민센터', 0.8362565),
 ('주민 단체 재단 사랑 사람들 봉사 나눔 마을 기부 행복', 0.8223524),
 ('제도 방안지 건축 기획안 하늘색 사회공헌 ppt 보고 행사 기업', 0.7856094),
 ('모집 시골 자원봉사자 봉사 농촌 봉사활동', 0.78497916),
 ('제안발표 애니메이션 도시 주민자치 토론 정책참여 주민참여 테스트', 0.774582)]

In [ ]:
sentence = '''
제목: 세계를 변화시킬 IT 기업에서 함께 일해보세요!

소제목: 혁신과 기술로 미래를 연결하는 IT 전문가를 찾습니다.

첫 번째 단락:

우리는 세계적으로 인정받는 IT 기업으로서 혁신과 기술 개발에 주력하고 있습니다.
우리의 목표는 사람들의 삶을 편리하게 만들고, 사회와 비즈니스 영역에서 긍정적인 변화를 이끄는 것입니다.
우리는 열정적이고 창의적인 IT 전문가를 찾고 있습니다.
두 번째 단락:

우리 회사에서는 최신 기술을 활용하여 다양한 프로젝트를 진행합니다.
클라우드 컴퓨팅, 인공지능, 빅데이터 분석 등의 분야에서 우수한 역량을 보유한 전문가들과 함께 일합니다.
우리는 협업과 지속적인 학습을 중요시하며, 다양한 기회를 제공합니다.
세 번째 단락:

우리는 직원들의 성장과 발전을 위해 노력합니다.
경력 개발 프로그램과 교육 기회, 학습 자료 등을 제공하여 전문 기술과 리더십을 강화합니다.
우수한 성과에는 공정하고 경쟁력 있는 보상과 인센티브가 제공됩니다.
마지막 단락:

우리는 다양성과 포용성을 존중하는 기업문화를 갖추고 있습니다.
팀워크와 즐거움을 추구하며, 긍정적인 근무 환경을 조성합니다.
우리와 함께 미래를 형성하고, 세계를 변화시킬 일에 참여해보세요!
'''
get_top_keyword(sentence, 10)

[('효율 컴퓨터 어플 생산성 협업 직장 조직 스마트 업무효율 업무 회사 핸드폰 스마트폰 프로그램 툴 비즈니스', 0.91818523),
 ('과정 직장 코딩 회사원 기술 심플 자기계발 강좌 직원 배움 비비드 역량 개발 성장 강연 동료 간단 회사 강의 리뷰 스킬 온라인 조직 직무 디자인 프로그램 비즈니스 후기 컬러 뱃지 교육 커리어 도형',
  0.91681373),
 ('직장인 엑셀 교육 수업 민트 온라인강의 파랑 자기계발 강의', 0.8731241),
 ('세미나 단순 소프트웨어 컨퍼런스 직장 코딩 기술 사내 회의 심플 사전등록 코드 혁명 발표 개발 회사 데이터 세션 학회 4차 기업 타이포그라피 분석 추상적인 미래 비즈니스 사회 사업 개발자 학술대회',
  0.87094563),
 ('미디어 그라데이션 IT 영상 직장 유튜브 CEO 아이티 회사 정보 그래픽 강의 기업', 0.842921),
 ('교육담당자 파른색피피티 cs교육 cs', 0.82483804),
 ('컴퓨터 비즈니스 소프트웨어 it 개발 강연 전자 코딩 개발자 강의', 0.8187762),
 ('컬러 테두리 탐방보고서 직업 인터뷰', 0.8119366),
 ('비누방울 공고 3d 타이포그래피 전문적인 비비드 it 개발 채용 작가 코딩 컬러풀 직업 개발자 투명구슬 직종', 0.8071914),
 ('컴퓨터 고급언어 it 저급언어 업무 프로그래밍 개발자 개발기본용어 프로그래밍언어 개념', 0.79775107)]

In [ ]:
sentence = '''
제목: 안전한 수영을 위한 주의사항!

소제목: 모두가 즐거운 수영을 위해 지켜야 할 사항들

첫 번째 단락:

수영장을 이용하실 때에는 반드시 안전을 최우선으로 생각해야 합니다.
다음은 수영장 이용시 지켜야 할 중요한 주의사항입니다.
두 번째 단락:

수영 전에는 몸을 충분히 풀어주세요.
워밍업과 스트레칭을 통해 근육을 준비하고 부상을 예방하세요.
건강한 상태에서만 수영을 시작하세요.
세 번째 단락:

물놀이를 즐길 때에는 깊은 수영장에서는 반드시 보호자의 도움을 받아야 합니다.
신체적인 능력에 맞는 구명용품을 착용하고, 생명보호 기구의 위치를 알아두세요.
다른 수영객과 충돌하지 않도록 주의하세요.
네 번째 단락:

수영장 내부에서는 안전한 행동이 중요합니다.
풀에서 뛰어오르거나 미끄러지는 행동은 사고의 원인이 될 수 있으니 자제해주세요.
슬리퍼나 비닐 바닥에는 주의하고, 갈라진 타일 등 위험한 부분에서 멀리 떨어져서 수영하세요.
다섯 번째 단락:

세안을 위해 수영장 샤워 시설을 이용할 때에는 청결을 유지해주세요.
개인 위생을 지켜 모두가 건강하게 수영을 즐길 수 있도록 합시다.
마지막 단락:

안전한 수영은 모두의 책임입니다.
주위 환경과 다른 수영객을 배려하며 즐거운 수영을 즐기고, 안전에 최선을 다해주세요.
'''
get_top_keyword(sentence, 10)

[('영법 배영 atoz 효과 안전수칙 유산소 크롤 물 수영 평영 접영', 0.95074356),
 ('영법 수영장 교육 토끼 어린이 물 바다 수영 수중 여름 카드뉴스', 0.91379094),
 ('수영 심플 여름 주의사항 라인 스포츠센터 건강 다이어트 소셜미디어 운동 강조 활동적인 수영장 플랫한 아이콘 하늘색 분홍 친근한 일러스트 미니멀 가로 민트 시원한 물 pop',
  0.89995444),
 ('물방울 수영레슨 건강 수영장 일러스트 정사각형 다이어트 스포츠센터 아이스크림 그림 귀여운 아기자기한 블루 운동 시원한 물 수영 여름',
  0.8830756),
 ('수영 심플 여름 주의사항 라인 화이트 스포츠센터 건강 다이어트 소셜미디어 텍스트 운동 강조 활동적인 수영장 플랫한 하늘색 분홍 친근한 일러스트 미니멀 가로 시원한 물 pop',
  0.87061775),
 ('스포츠 세로 연두 해변 심플 여름 라인 노랑 튜브 원형 강조 파랑 수영장 젊은 다이내믹 레저 웹포스터 일러스트 그림 재밌는 다이빙 물 시원한 아트웍',
  0.8668122),
 ('스포츠 손그림 설명 감성 공지 구름 스쿠버다이빙 안전 안내 바다 레저 서핑 인스타그램 여가 해양 일러스트 교육 다이빙 게시물 해양레저',
  0.86643934),
 ('스포츠 심플 목록 주황 ppt 모던 설명 귀여운 깔끔 파랑 공지 스쿠버다이빙 3d 안전 안내 아이콘 바다 레저 여가 프레젠테이션 흰색 일러스트 미니멀 가로 교육 재밌는 다이빙 해양레저 그래프 도형',
  0.86368424),
 ('그라데이션 스포츠 배색 흑백 여름 설명 감성 공지 스쿠버다이빙 안전 안내 바다 레저 여가 해양 교육 다이빙 해양레저 도형',
  0.8634212),
 ('스포츠 수중탐사 수영 여름 ppt 설명 공지 스쿠버다이빙 잠수 안전 안내 바다 레저 여가 해저 프레젠테이션 해양 교육 스킨스쿠버 다이빙 해양레저',
  0.8623167)]

In [ ]:
sentence = """
Title: Safety Guidelines for Enjoying Swimming!

Subtitle: Important precautions to ensure a safe and enjoyable swimming experience.

First paragraph:

When using a swimming pool, safety should be your top priority.
Here are some important precautions to keep in mind while using the swimming pool.
Second paragraph:

Always warm up your body before swimming.
Perform warm-up exercises and stretches to prepare your muscles and prevent injuries.
Start swimming only when you are in good health.
Third paragraph:

When engaging in water activities, especially in deep pools, always seek the assistance of a guardian.
Wear appropriate flotation devices according to your swimming ability and be aware of the location of life-saving equipment.
Be cautious and avoid collisions with other swimmers.
Fourth paragraph:

Practice safe behavior within the swimming pool area.
Avoid jumping or diving into the pool and refrain from engaging in activities that may lead to accidents.
Be mindful of slippery surfaces, such as tiles or wet floors, and maintain a safe distance from hazardous areas while swimming.
Fifth paragraph:

Maintain cleanliness while using the shower facilities in the swimming pool for personal hygiene.
By practicing good hygiene, we can ensure a healthy swimming environment for everyone.
Final paragraph:

Ensuring a safe swimming experience is everyone's responsibility.
Let's be considerate of our surroundings, respect other swimmers, and prioritize safety to enjoy a pleasant swimming experience.
"""
get_top_keyword(sentence, 10)

[('스포츠 심플 순수한 모던 설명 귀여운 자연적인 개인 깔끔 취미 잔잔한 파란 캘리그라피 하늘색 레저 여가 키치 일러스트 다이빙 요트 카드뉴스',
  0.9701153),
 ('스포츠 손그림 설명 감성 공지 구름 스쿠버다이빙 안전 안내 바다 레저 서핑 인스타그램 여가 해양 일러스트 교육 다이빙 게시물 해양레저',
  0.9645591),
 ('스포츠 분할 연두 심플 파란색 줄 라인 설명 개인 취미 젊은 하늘색 레저 여가 선 섬세한 다이빙 요트 클래식 도형 감성적인 카드뉴스',
  0.9521321),
 ('뉴스 안전수칙 안전 바다 여행 낚시 안내 수상스키 인스타 페북 레저 보트 휴가 정보 홍보 해변 소셜미디어 여름휴가 제트스키 마케팅 해수욕장 페이스북 SNS 광고 서핑 여름 카드뉴스 레져 인스타그램 수상레저',
  0.94891435),
 ('스포츠 세로 연두 해변 심플 여름 라인 노랑 튜브 원형 강조 파랑 수영장 젊은 다이내믹 레저 웹포스터 일러스트 그림 재밌는 다이빙 물 시원한 아트웍',
  0.9398216),
 ('크루 쓰레기 해변 보호 모집 여자 캐릭터 귀여운 감성 에코 꽃게 나무 환경 바다 인스타그램 게 비치코밍 컬러 일러스트 해수욕장 동아리 색감',
  0.937271),
 ('펜 스포츠 마케팅 세로 맑은 해변 심플 여름 텍스트 자연적인 강조 홍보 파스텔 하늘색 바다 레저 서핑 웹포스터 흰색 그림 야자수 컬러풀 자유로운 시원한 물 아트웍',
  0.9371689),
 ('그라데이션 스포츠 배색 흑백 여름 설명 감성 공지 스쿠버다이빙 안전 안내 바다 레저 여가 해양 교육 다이빙 해양레저 도형',
  0.93669105),
 ('스포츠 정사각형 수영 심플 물놀이 여름 모던 라인 노랑 소셜미디어 귀여운 텍스트 개인 취미 파랑 활동적인 홍보 수영장 강습 하늘색 프레임 여가 일러스트 가로 수영강사 자유로운 물',
  0.9363587),
 ('방학 썸머 수영 서핑 여름', 0.9340149)]

In [ ]:
sentence = '''
공부
'''
get_top_keyword(sentence, 10)

[('공부 학용품 안내 모음집 하늘색 필기구 파란색 핑크색 분홍색', 0.9642383),
 ('썸네일 친구 친구유형 신학기 유튜브 학생 새학기 노트', 0.9589652),
 ('면접팁 면접백과 취업 면접자 남색 면접', 0.95884514),
 ('선배 대학교 학기 족보 학교생활 개강 동아리 신입생 과제 시험 후배 새학기 꿀팁 등교', 0.958485),
 ('칠판 얼굴평가 친구 평가 친구들 입학 인기 중학생 미모 학생 얼평 새학기 얼굴 뷰티 교실 고등학생', 0.95828795),
 ('인싸 친구 걱정 학교 아싸 비법 학생 고민 새학기 교실', 0.95571965),
 ('교육 블루 취업 대기업 하늘색 회사 z세대 비즈니스 카드뉴스', 0.9550811),
 ('흥미 청소년 중학생 장래희망 학생 직업 순위 남녀 고등학생', 0.9550407),
 ('사이드프로젝트 직장인 부업 인스타피드 취미 인스타그램', 0.95412123),
 ('사이드프로젝트 엔잡러 직장인 사이드잡 투잡 n잡', 0.9526775)]

In [ ]:
%cd /content/drive/MyDrive/model